# Clustering and PCA

### Mushroom Dataset

Podeis obtener el conjunto de datos en el siguiente enlace:

[Mushroom Dataset](https://www.kaggle.com/uciml/mushroom-classification)

Como podréis comprobar, hay muchas variables, todas ellas categóricas, por lo que exploraciones con scatterplot no nos serán útiles como en otros casos.

La variable a predecir ``class`` es binaria.


In [16]:
# Carga de librerías, las que hemos considerado básicas, añadid lo que queráis :)

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [17]:
import pandas as pd
import os

def load_data(file_path):
    """
    Carga los datos del archivo CSV.

    Args:
        file_path (str): Ruta al archivo CSV.

    Returns:
        pd.DataFrame: DataFrame con los datos cargados o None si hay un error.
    """

    # Obtener la ruta absoluta del archivo para evitar problemas con rutas relativas
    abs_file_path = os.path.abspath(file_path)

    # Verificar si el archivo existe en la ruta especificada
    if not os.path.exists(abs_file_path):
        # Imprimir mensajes de error detallados si el archivo no existe
        print(f"Error: El archivo no existe en la ruta: {abs_file_path}")
        print(f"Directorio actual: {os.getcwd()}")  # Mostrar el directorio actual para ayudar a depurar
        print("Contenido del directorio:")
        try:
            # Intentar listar el contenido del directorio para ayudar a identificar el problema
            print(os.listdir(os.path.dirname(abs_file_path)))
        except FileNotFoundError:
            print("El directorio no existe.")  # Informar si el directorio tampoco existe
        return None  # Devolver None para indicar que la carga falló

    try:
        # Intentar cargar los datos utilizando pandas
        df = pd.read_csv(abs_file_path)
        print(f"Datos cargados exitosamente. Shape: {df.shape}")  # Confirmar la carga exitosa
        return df
    except Exception as e:
        # Capturar cualquier excepción que ocurra durante la carga e imprimir un mensaje de error
        print(f"Error al cargar los datos: {e}")
        return None



In [18]:
# Bloque de ejemplo de uso (solo se ejecuta si el script se ejecuta directamente)
if __name__ == "__main__":
    # Obtener la ruta al archivo CSV dentro de la estructura del proyecto
    #script_dir = os.path.dirname(os.path.abspath(__file__)) #sólo sirve cuando es .py porque la variable __file__ no está disponible porque no se está ejecutando un script de Python directamente.
    #project_root = os.path.dirname(os.path.dirname(script_dir))
    #data_path = os.path.join(project_root, "data", "mushrooms.csv") #(project_root,"carpeta","subcarpeta","archivo.csv")
    data_path = data_path = os.path.join(os.getcwd(), "data", "mushrooms.csv")

    print(f"Intentando cargar el archivo desde: {data_path}")


    # Llamar a la función load_data para cargar los datos
    
    df = load_data(data_path)
    if df is not None:
        # Imprimir las primeras filas del DataFrame si la carga fue exitosa
        print(df.head())
    else:
        # Imprimir un mensaje de error si la carga falló
        print("No se pudieron cargar los datos. Verifica la ruta del archivo y su existencia.")

Intentando cargar el archivo desde: c:\4_F5\012_noSupervisado\Nat_noSupervisado\data\mushrooms.csv
Datos cargados exitosamente. Shape: (8124, 23)
  class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stal

### Leer conjunto de datos y primer vistazo

In [19]:
# Leer el csv y sacar por pantalla las cinco primeras filas.

def lectura_csv(df):
    return df.head()
print(lectura_csv(df))

  class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w         p          w   
1       

La librería OS nos permite navegar entre los directorios del sistema

### Exploración de datos

In [20]:
# Descripción del conjunto de datos, estándard.


print(df.describe(include='object').T) # con el parámetro include='object' para obtener estadísticas descriptivas para las columnas categóricas.

                         count unique top  freq
class                     8124      2   e  4208
cap-shape                 8124      6   x  3656
cap-surface               8124      4   y  3244
cap-color                 8124     10   n  2284
bruises                   8124      2   f  4748
odor                      8124      9   n  3528
gill-attachment           8124      2   f  7914
gill-spacing              8124      2   c  6812
gill-size                 8124      2   b  5612
gill-color                8124     12   b  1728
stalk-shape               8124      2   t  4608
stalk-root                8124      5   b  3776
stalk-surface-above-ring  8124      4   s  5176
stalk-surface-below-ring  8124      4   s  4936
stalk-color-above-ring    8124      9   w  4464
stalk-color-below-ring    8124      9   w  4384
veil-type                 8124      1   p  8124
veil-color                8124      4   w  7924
ring-number               8124      3   o  7488
ring-type                 8124      5   

In [21]:
# Información sobre el tipo de datos de cada feature.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [23]:
# Igual que otras veces, una linea, contar los nulos por variable.

In [15]:
def count_missing(df):
    missing_values = df.isnull().sum()
    if (missing_values == 0).all():
        return "El conteo por columna de valores 'missing' resulta 0. Es decir, no hay valores 'missing'"
    else:
        return missing_values

print(count_missing(df))

El conteo por columna de valores 'missing' resulta 0. Es decir, no hay valores 'missing'


#### Buscar valores extraños. Para ello, ver los valores únicos en cada feature

In [ ]:
# Obtener un nuevo dataframe de dos columnas donde en la primera estén las features (features) y en la otra los valores únicos
# asociados (n_values).


#### Tratar aquellos valores que entendamos que sean nulos


In [ ]:
# Imputaciones. Podéis quitar esos puntos (fila entera), imputar con la moda o dejar ese valor como una posibilidad más.

#### Mirad cuántos valores hay en cada feature, ¿Todas las features aportan información? Si alguna no aporta información, eliminadla

In [ ]:
# Dejar por el camino si procede.

#### Separar entre variables predictoras y variables a predecir

In [ ]:
# La variable que trata de predecir este conjunto de datos es 'class'.
y =
X =

#### Codificar correctamente las variables categóricas a numéricas

In [ ]:
# One Hot Encoder (una linea).

#### Train test split

In [ ]:
# Os lo dejamos a todos igual
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## PCA

Es un conjunto de datos del que aún no hemos visto nada (no tenemos graficas) así que vamos a hacer algunas. Tenemos el problema de que son muchas variables, **PCA al rescate**: le pedimos que nos de dos dimensiones y las pintamos, sabemos que serán **aquellas que retengan más información**.

In [ ]:
pca =       # metodo de sklearn
pca.fit(X_train)

# Representar en un scatterplot y poner en color las etiquetas de entrenamiento

Parece que está bastante separadito, parece que a ojo mucho se puede ver :)

Igualmente, vamos a entrenar un clasificador a ver qué tal lo hace antes de editar más

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Definir el clasificador y el número de estimadores
# 2. Entrenar en train
# 3. Calcular la precisión sobre test

Es un conjunto sencillo y Random Forest es muy bueno en su trabajo, Igualmente, vamos a ver qué tamaño tenemos de dataset:


In [ ]:
X_train.shape

¿Muchas features no? Vamos a reducir las usando PCA.

In [ ]:
n_features = # definir un rango de valores a probar
scores = []

for n in n_features:

    # Hacer PCA sobre X_train
    # 1. Definir PCA
    # 2. Aprender PCA sobre X_train

    # Entrenar Random Forest
    # 1. Definir el RF
    # 2. Entrenar clasificador

    # Guardar el score


sns.lineplot(x=n_features, y=scores)


Vale, estamos viendo que a partir de unas 10 features ya tenemos el score que queríamos y además hemos reducido las variables a un 10% de las que teníamos, incluso menos que las variables originales.

## Clustering

Viendo que el conjunto de datos es sencillito, podemos intentar hacer algo de clustering a ver qué información podemos obtener.

El primer paso va a ser importar la función de Kmeans de sklearn, y a partir de ahi, vamos a buscar el valor óptimo de clusters. Como hemos visto anteriormente, este valor lo obtenemos, por ejemplo, del codo de la gráfica que representa el total de las distancias de los puntos a los centros de los clusters asociados. Os dejo la página de la documentación de sklearn para que lo busquéis:

[K-Means on sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

Con esto solo hay que ahora generar los modelos de kmeans, evaluar y pintar la gráfica para los valores de ``k`` que establezcais.




In [ ]:
from sklearn.cluster import KMeans

scores = []
k_values = # definir un rango
for a in k_values:

    # Definir Kmeans y ajustar
    # Guardar la predicción

sns.lineplot(x=k_values, y=scores)

Con el valor que hayáis obtenido de la gráfica, podéis obtener una buena aproximación de Kmeans y con ello podemos pasar a explorar cómo de bien han separado la información los distintos clusters. Para ello, se va a hacer un ``catplot``, seaborn os lo hará solito. Con esto lo que se pretende ver es la distribución de la varaible a predecir en función del cluster que haya determinado Kmeans.

In [ ]:
# Aprender Kmeans con el valor de K obtenido.

kmeans = # Definir y entrenar Kmeans.

# Preparar el catplot.


# Pintar.
ax = sns.catplot(col=, x=, data=, kind='count',col_wrap=4)

Vamos a ver qué tal queda esto pintado. Para ello, repetimos el scatterplot de antes pero usando como color el cluster asignado por kmeans.

In [ ]:
# Entrenar PCA para representar.

# Usar un color por cada cluster.


¿Es bastante parecido no? No es tan bueno como el Random Forest, pero ha conseguido identificar bastante bien los distintos puntos del dataset sin utilizar las etiquetas. De hecho, el diagrama de factor que hemos visto antes muestra que solo un par de clusters son imprecisos. Si no hubieramos tenido etiquetas esta aproximacion nos hubiera ayudado mucho a clasificar los distintos tipos de hongos.